In [2]:
##Imported Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
##import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")


In [1]:
##from google.colab import files
##uploaded = files.upload()

In [4]:
#import io
#df =pd.read_excel(io.BytesIO(uploaded['Data_Train.xlsx']))
df=pd.read_excel('Data_Train.xlsx')

In [5]:
df.head()

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,31103489027986,26-04-2010,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,10088999935915,21-06-2009,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,40148966010272,01-05-2013,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6
3,37157708563676,28-09-2015,20825,20007,19115,RESTAURANT,3661,id-11235901,id_1890134,81859.0,CANVASS RE-INSPECTION,31.0,Medium,locid11839,4
4,47478049564374,09-12-2015,2136,16867,10409,RESTAURANT,7876,id-11235901,id_1890134,81886.0,COMPLAINT,30.0,High,locid12264,4


In [6]:
df.Type.value_counts()
df.columns
df.isna().sum()
df1= df.dropna()
df1.head()

df1.shape
## Since ID,Date does not looks like they are needed dropping from DF
df2=df1.drop(['ID','Date'],axis=1)
df2.shape
len(df2.Type.unique())
type_stats = df2.groupby('Type')['Type'].agg('count').sort_values(ascending=False)
len(type_stats[type_stats<=40])
type_stats_less_than_50 = type_stats[type_stats<=50]
type_stats_less_than_50.shape
type_stats_less_than_50.head(4)
df2.Type = df2.Type.apply(lambda x: 'other' if x in type_stats_less_than_50 else x)
len(df2.Type.value_counts())
df2.shape
df2.Type.value_counts()
len(df2.Geo_Loc.value_counts())
geo_stats = df2.groupby('Geo_Loc')['Geo_Loc'].agg('count').sort_values(ascending=False)
len(geo_stats)
len(geo_stats[geo_stats<=40])
geo_stats_less_than_40 = geo_stats[geo_stats<=40]
geo_stats_less_than_40.shape
df2.Geo_Loc = df2.Geo_Loc.apply(lambda y: 'OTHER' if y in geo_stats_less_than_40 else y)
len(df2.Geo_Loc.value_counts())
len(df2.Type.value_counts())
df2.Geo_Loc.value_counts()

OTHER         101601
locid16551      1640
locid16811       373
locid3229        300
locid7295        266
locid7485        241
locid5317        192
locid7590        177
locid1884        163
locid7988        158
locid8358        148
locid4664        131
locid7944        131
locid8348        116
locid8308        107
locid6834        106
locid7838        104
locid5696         95
locid6838         93
locid13219        84
locid8766         79
locid6769         76
locid9134         72
locid7828         69
locid8069         67
locid7607         67
locid7984         66
locid7433         66
locid3576         64
locid5994         63
locid7430         62
locid7682         62
locid8351         62
locid8648         61
locid6705         61
locid6246         59
locid6736         59
locid7762         58
locid8955         57
locid2313         55
locid9102         53
locid9043         50
locid7591         48
locid1158         46
locid7287         46
locid6442         46
locid7101         46
locid9608    

In [7]:
df2.head()

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,OTHER,4
1,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,OTHER,4
3,20825,20007,19115,RESTAURANT,3661,id-11235901,id_1890134,81859.0,CANVASS RE-INSPECTION,31.0,Medium,OTHER,4
4,2136,16867,10409,RESTAURANT,7876,id-11235901,id_1890134,81886.0,COMPLAINT,30.0,High,OTHER,4
5,13009,7942,7547,RESTAURANT,12275,id-11235901,id_1890134,81877.0,CANVASS,18.0,High,OTHER,1


In [8]:
df2.shape

(108281, 13)

In [9]:
df2['Geo_Loc'].value_counts()

OTHER         101601
locid16551      1640
locid16811       373
locid3229        300
locid7295        266
locid7485        241
locid5317        192
locid7590        177
locid1884        163
locid7988        158
locid8358        148
locid4664        131
locid7944        131
locid8348        116
locid8308        107
locid6834        106
locid7838        104
locid5696         95
locid6838         93
locid13219        84
locid8766         79
locid6769         76
locid9134         72
locid7828         69
locid8069         67
locid7607         67
locid7984         66
locid7433         66
locid3576         64
locid5994         63
locid7430         62
locid7682         62
locid8351         62
locid8648         61
locid6705         61
locid6246         59
locid6736         59
locid7762         58
locid8955         57
locid2313         55
locid9102         53
locid9043         50
locid7591         48
locid1158         46
locid7287         46
locid6442         46
locid7101         46
locid9608    

In [9]:
df2.shape

(108281, 13)

In [10]:
df3 = pd.get_dummies(df2.Type)

In [11]:
df3.shape

(108281, 25)

In [12]:
df4 = pd.get_dummies(df2, columns=['Type'],prefix = "",prefix_sep= "")

In [13]:
df5 = df4.drop(['other'],axis=1)

In [14]:
df5.shape

(108281, 36)

In [15]:
df6 = pd.get_dummies(df5, columns=['City','State','Reason'],prefix = "",prefix_sep= "")

In [16]:
df6.shape

(108281, 53)

In [17]:
df7 = pd.get_dummies(df6, columns=['RiskLevel'],prefix = "",prefix_sep= "")

In [18]:
df8 = pd.get_dummies(df7, columns=['Geo_Loc'],prefix = "",prefix_sep= "")

In [19]:
df8.shape

(108281, 111)

In [20]:
df9 = df8.drop(['OTHER'],axis=1)

In [27]:
df9.shape

(108281, 110)

In [28]:
df10=df9.copy()

In [29]:
df10.shape

(108281, 110)

In [30]:
X = df9.drop(['Inspection_Results'],axis='columns')
X.head(2)

,LicenseNo,FacilityID,FacilityName,Street,LocationID,SectionViolations,BAKERY,BANQUET HALL,CATERING,CHILDREN'S SERVICES FACILITY,...,locid8348,locid8351,locid8358,locid8648,locid8766,locid8955,locid9043,locid9102,locid9134,locid9608
0,4744,8123,7715,15522,81876.0,33.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2973,12268,11664,3057,81862.0,33.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df10['Inspection_Results'].replace(6,1,inplace=True)
df10['Inspection_Results'].replace(3,5,inplace=True)
df10['Inspection_Results'].replace(2,5,inplace=True)

In [31]:
df10['Inspection_Results'].value_counts()

4    61472
1    26174
5    20256
3      314
2       39
6       26
Name: Inspection_Results, dtype: int64

In [32]:
X = df10.drop(['Inspection_Results'],axis='columns')
y=df10.Inspection_Results

In [33]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [36]:
lnr_class = LogisticRegression()
lnr_class.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
from sklearn.model_selection import cross_val_score
cross_val_score(lnr_class, X_train, y_train, cv=5, scoring='accuracy').mean()

0.8109005592388735

In [38]:
y_predict = lnr_class.predict(X_test)
accuracy_score(y_test, y_predict)

0.8133600123133754

In [39]:
pd.crosstab(y_test, y_predict)

col_0,1,4,5
Inspection_Results,,,
1,4560,1131,2214
2,3,4,5
3,44,38,6
4,112,18253,27
5,2015,455,3609
6,2,6,1


In [33]:
## Training using SVC
# train the model on train set 
#model = SVC() 
#model.fit(X_train, y_train) 
  
# print prediction results 
predictions = model.predict(X_test) 
print(classification_report(y_test, predictions)) 

NameError: ignored

In [0]:
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
#grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
#grid.fit(X_train, y_train)

In [40]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [41]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.15%


In [42]:
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)
pd.crosstab(y_test, y_predict)

col_0,1,4,5
Inspection_Results,,,
1,5826,210,1869
2,3,3,6
3,40,30,18
4,111,18229,52
5,1326,172,4581
6,3,4,2


In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

array([[ 5826,     0,     0,   210,  1869,     0],
       [    3,     0,     0,     3,     6,     0],
       [   40,     0,     0,    30,    18,     0],
       [  111,     0,     0, 18229,    52,     0],
       [ 1326,     0,     0,   172,  4581,     0],
       [    3,     0,     0,     4,     2,     0]], dtype=int64)

In [0]:
# Let's implement simple classifiers

from sklearn import ensemble

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [39]:
# Wow our scores are getting even high scores even when applying cross validation.
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

Classifiers:  LogisticRegression Has a training score of 73.0 % accuracy score
Classifiers:  KNeighborsClassifier Has a training score of 56.99999999999999 % accuracy score
Classifiers:  RandomForestClassifier Has a training score of 87.0 % accuracy score
Classifiers:  DecisionTreeClassifier Has a training score of 85.0 % accuracy score


In [43]:
# print prediction results 
predictions = classifier.predict(X_test) 
print(classification_report(y_test, predictions)) 

              precision    recall  f1-score   support

           1       0.73      0.73      0.73      7914
           4       0.98      0.98      0.98     18392
           5       0.64      0.64      0.64      6179

    accuracy                           0.85     32485
   macro avg       0.78      0.78      0.78     32485
weighted avg       0.85      0.85      0.85     32485



In [45]:
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(n_estimators=50, silent=True)
lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=5, verbose=False)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=50, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
y_pred = model.predict(X_test)

In [49]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.15%


In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]



In [42]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.07%


In [0]:
# Use GridSearchCV to find the best parameters.
from sklearn.model_selection import GridSearchCV


# Logistic Regression 
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}



grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)
# We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_

knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(X_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_

# Support Vector Classifier
##svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
##grid_svc = GridSearchCV(SVC(), svc_params)
##grid_svc.fit(X_train, y_train)

# SVC best estimator
##svc = grid_svc.best_estimator_

# DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))}
grid_tree = GridSearchCV(RandomForestClassifier(), tree_params)
grid_tree.fit(X_train, y_train)

# tree best estimator
tree_clf = grid_tree.best_estimator_

In [55]:
print(tree_clf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [50]:
print(knears_neighbors )
print(tree_clf)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [1]:
model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

NameError: name 'RandomForestClassifier' is not defined

In [57]:
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)
pd.crosstab(y_test, y_predict)

col_0,1,4
Inspection_Results,,
1,688,7226
4,3,18389
5,79,6100


In [58]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 58.73%


In [44]:
######## SMOTE for Balancing the dataset target values
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote,y_train_smote = smote.fit_sample(X_train.astype(float),y_train)
from collections import Counter
print('Before SMOTE',Counter(y_train))
print('After SMOTE',Counter(y_train_smote))

Before SMOTE Counter({4: 43080, 1: 18286, 5: 14430})
After SMOTE Counter({1: 43080, 4: 43080, 5: 43080})


In [0]:
# fit model no training data
model = RandomForestClassifier(n_estimators=20,random_state=101)
model.fit(X_train_smote, y_train_smote)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [46]:
y_predict = model.predict(X_test)
accuracy_score(y_test, y_predict)
pd.crosstab(y_test, y_predict)

col_0,1,4,5
Inspection_Results,,,
1,5886,174,1854
4,106,18163,123
5,1866,207,4106


In [47]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.67%


In [0]:
# Create a training and testing dataset
# Modeling
import lightgbm as lgb


train_set = lgb.Dataset(data = X_train, label = y_train)
test_set = lgb.Dataset(data = X_test, label = y_test)
N_FOLDS = 5
MAX_EVALS = 5

In [0]:
model = lgb.LGBMClassifier()
default_params = model.get_params()

# Remove the number of estimators because we set this to 10000 in the cv call
del default_params['n_estimators']

# Cross validation with early stopping
cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, 
                    metrics = 'auc', nfold = N_FOLDS, seed = 42)

In [0]:
print('The maximum validation ROC AUC was: {:.5f} with a standard deviation of {:.5f}.'.format(cv_results['auc-mean'][-1], cv_results['auc-stdv'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

The maximum validation ROC AUC was: 1.00000 with a standard deviation of 0.00000.
The optimal number of boosting rounds (estimators) was 1.


In [0]:

# Optimal number of esimators found in cv
model.n_estimators = len(cv_results['auc-mean'])

# Train and make predicions with model
model.fit(X_train, y_train)
preds = model.predict_proba(y_test)[:, 1]
baseline_auc = roc_auc_score(y_test, preds)

print('The baseline model scores {:.5f} ROC AUC on the test set.'.format(baseline_auc))

ValueError: ignored

In [0]:
def objective(hyperparameters, iteration):
    """Objective function for grid and random search. Returns
       the cross validation score from a set of hyperparameters."""
    
    # Number of estimators will be found using early stopping
    if 'n_estimators' in hyperparameters.keys():
        del hyperparameters['n_estimators']
    
     # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, X_train, num_boost_round = 10000, nfold = N_FOLDS, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = 42)
    
    # results to retun
    score = cv_results['auc-mean'][-1]
    estimators = len(cv_results['auc-mean'])
    hyperparameters['n_estimators'] = estimators 
    
    return [score, hyperparameters, iteration]


In [0]:
model = lgb.LGBMModel()
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [0]:
param_grid =  {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}


In [0]:
import random
random.seed(50)

# Randomly sample a boosting type
boosting_type = random.sample(param_grid['boosting_type'], 1)[0]

# Set subsample depending on boosting type
subsample = 1.0 if boosting_type == 'goss' else random.sample(param_grid['subsample'], 1)[0]

print('Boosting type: ', boosting_type)
print('Subsample ratio: ', subsample)

TypeError: ignored